In [1]:
import numpy as np
import pandas as pd

In [2]:
T = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
col = ['Size', 'BM', 'OP', 'INV']

In [3]:
for t in T:
    df = pd.read_csv('../data/step2/value/{}.csv'.format(t))
    for c in col:
        a_20 = df[c].quantile(0.2)
        a_40 = df[c].quantile(0.4)
        a_60 = df[c].quantile(0.6)
        a_80 = df[c].quantile(0.8)
        for i in range(df.shape[0]):
            if df.loc[i, c] < a_20:
                df.loc[i, c] = 'A'
            elif df.loc[i, c] < a_40:
                df.loc[i, c] = 'B'
            elif df.loc[i, c] < a_60:
                df.loc[i, c] = 'C'
            elif df.loc[i, c] < a_80:
                df.loc[i, c] = 'D'
            else:
                df.loc[i, c] = 'E'
    df.to_csv('../data/analyze55/label/{}.csv'.format(t))
    

In [4]:
finance = pd.read_csv('../data/step1/finance.csv')
returns = pd.read_csv('../data/step1/returns.csv')
for df in [finance, returns]:
    df['Time'] = pd.to_datetime(df['Time'])

In [5]:
stk_group = []
for t in T:
    df = pd.read_csv('../data/analyze55/label/{}.csv'.format(t))
    for label1 in ['A', 'B', 'C', 'D', 'E']:
        for item in ['BM', 'OP', 'INV']:
            for label2 in ['A', 'B', 'C', 'D', 'E']:
                stk = df[(df['Size'] == label1) & (df[item] == label2)]['Stkcd'].tolist()
                stk_group.append((str(t) + '_' + item + '_' + label1 + label2, stk))
stk_group = dict(stk_group)

In [6]:
R = []
for x in stk_group:
    [year, item, label] = x.split('_')
    year = int(year)
    m = 7
    
    df = pd.DataFrame(index=None)
    for st in stk_group[x]:
        tmp = returns[returns['Stkcd'] == st]
        df = pd.concat([df, tmp])
    
    for y in [year - 2, year - 1, year]:
        while (y < year and m <= 12) or (y == year and m <= 7):
            r = df[(df['Time'].dt.year == y) & (df['Time'].dt.month == m)][['Stkcd', 'Msmvosd', 'Mretwd']]
            r['S*R'] = r['Msmvosd'] * r['Mretwd']
            ans = r['S*R'].sum() / r['Msmvosd'].sum()
            R.append((str(year) + '_' + str(y) + str(m) + '_' + item + '_' + label, ans))
            m += 1
        m = 1
        
R = pd.DataFrame(R, index=None)
R.columns = ['label', 'R']
R.to_csv('../data/analyze55/R.csv')

In [7]:
d = []
R = pd.read_csv('../data/analyze55/R.csv', index_col=0)
Y = [2014, 2015, 2016]

for year in Y:
    print(year - 2, '~', year)
    for c in ['BM', 'OP', 'INV']:
        print(c)
        
        for label1 in ['A', 'B', 'C', 'D', 'E']:
            v1 = 0
            v2 = 0
            v3 = 0
            v4 = 0
            v5 = 0
            m = 7
            for y in [year - 2, year - 1, year]:
                while (y < year and m <= 12) or (y == year and m <= 7):
                    s = str(year) + '_' + str(y) + str(m) + '_' + c + '_'
                    v1 += R[R['label'] == s + 'A' + label1]['R'].values
                    v2 += R[R['label'] == s + 'B' + label1]['R'].values
                    v3 += R[R['label'] == s + 'C' + label1]['R'].values
                    v4 += R[R['label'] == s + 'D' + label1]['R'].values
                    v5 += R[R['label'] == s + 'E' + label1]['R'].values
                    m += 1
                m = 1

            v1 = '%.2f'%(v1 * 100 / 25) + '%'  
            v2 = '%.2f'%(v2 * 100 / 25) + '%'
            v3 = '%.2f'%(v3 * 100 / 25) + '%' 
            v4 = '%.2f'%(v4 * 100 / 25) + '%' 
            v5 = '%.2f'%(v5 * 100 / 25) + '%'
            print(v1, v2, v3, v4, v5) 
            d.append([v1, v2, v3, v4, v5])

d = pd.DataFrame(d)
d.to_csv('../data/analyze55/example.csv')

2012 ~ 2014
BM
2.42% 3.23% 3.38% 3.29% 2.50%
2.85% 1.62% 2.17% 2.06% 1.53%
1.92% 1.35% 1.99% 1.82% 1.05%
1.36% 1.16% 1.39% 0.98% 0.35%
0.92% 0.81% 0.64% 0.38% 0.64%
OP
1.91% 1.08% 1.76% 1.54% 0.55%
1.81% 1.61% 1.81% 1.48% 1.18%
2.05% 1.63% 2.27% 1.59% 0.68%
1.72% 2.13% 1.90% 1.69% 0.59%
1.98% 2.00% 2.04% 2.05% 1.08%
INV
1.94% 1.22% 1.99% 0.80% -0.23%
1.67% 1.31% 1.81% 1.67% 0.99%
1.70% 2.04% 1.76% 1.36% 0.48%
1.99% 1.14% 2.01% 2.07% 1.33%
2.20% 2.52% 2.25% 2.49% 1.87%
2013 ~ 2015
BM
5.25% 6.42% 6.73% 5.51% 4.71%
4.99% 4.52% 5.33% 5.41% 3.88%
4.26% 4.44% 5.01% 5.02% 4.16%
4.25% 4.79% 4.76% 4.40% 3.93%
3.91% 4.23% 3.80% 4.09% 3.56%
OP
4.55% 4.86% 4.91% 5.22% 6.05%
4.72% 4.94% 4.94% 5.00% 4.97%
4.72% 4.73% 5.64% 4.94% 5.15%
4.09% 4.80% 5.16% 4.91% 4.40%
4.87% 5.18% 5.33% 4.48% 3.07%
INV
4.55% 5.11% 5.70% 5.13% 4.62%
4.77% 4.42% 4.39% 4.56% 3.35%
4.56% 4.61% 4.96% 4.29% 3.36%
4.44% 4.75% 5.52% 4.75% 4.26%
4.65% 5.71% 5.53% 5.77% 4.91%
2014 ~ 2016
BM
5.79% 6.33% 6.17% 6.21% 5.10%
5.22% 4.90